In [18]:
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase.visualize import view

data = np.load("../EF/data-full.npz", allow_pickle=True)
keys = list(data.keys())
keys
R = data["R"]
P = data["P"]
Z = data["Z"]

In [12]:
def oriented_volume(p1, p2, p3, p4):
    """
    Oriented (signed) volume of tetrahedron (p1, p2, p3, p4).
    Returns OV = (1/6) * (p1-p4) · ((p2-p4) × (p3-p4))
    """
    p1 = np.asarray(p1, dtype=float)
    p2 = np.asarray(p2, dtype=float)
    p3 = np.asarray(p3, dtype=float)
    p4 = np.asarray(p4, dtype=float)

    a = p1 - p4
    b = p2 - p4
    c = p3 - p4

    return (1.0 / 6.0) * np.dot(a, np.cross(b, c))


def oriented_volume_det(p1, p2, p3, p4):
    """
    Same OV computed via the 4x4 determinant:
    OV = (1/6) * det([[1,1,1,1],
                     [x1,x2,x3,x4],
                     [y1,y2,y3,y4],
                     [z1,z2,z3,z4]])
    """
    p1 = np.asarray(p1, dtype=float)
    p2 = np.asarray(p2, dtype=float)
    p3 = np.asarray(p3, dtype=float)
    p4 = np.asarray(p4, dtype=float)

    M = np.array([
        [1,     1,     1,     1],
        [p1[0], p2[0], p3[0], p4[0]],
        [p1[1], p2[1], p3[1], p4[1]],
        [p1[2], p2[2], p3[2], p4[2]],
    ], dtype=float)

    return (1.0 / 6.0) * np.linalg.det(M)

In [19]:
R[0], Z[0]

(array([[[-0.856624, 0.129623, 1.934218],
         [-1.113226, 0.208861, 0.29253],
         [-2.395473, -0.129327, -0.244816],
         [-2.758218, 0.292541, -1.608939],
         [-3.369716, -0.937218, 0.535769],
         [0.00851, 0.750542, -0.561608],
         [0.074188, 2.289299, -0.495448],
         [1.530449, 2.646581, -0.349334],
         [2.25431, 1.399645, 0.131496],
         [1.32012, 0.278177, -0.140145],
         [1.720577, -1.093701, -0.088373],
         [0.86284, -2.237621, -0.873717],
         [3.046157, -1.479629, 0.505688],
         [-3.842935, 0.16159, -1.799584],
         [-2.564452, 1.387345, -1.717807],
         [-2.178615, -0.257288, -2.347192],
         [-2.88315, -1.865078, 0.871457],
         [-4.253185, -1.197732, -0.071191],
         [-3.687413, -0.336486, 1.41572],
         [-0.110561, 0.468629, -1.607865],
         [-0.39813, 2.77545, -1.388399],
         [-0.48387, 2.673344, 0.418778],
         [1.943412, 2.942095, -1.384198],
         [1.714216, 3.502091, 

In [20]:
atoms = ase.Atoms( Z[0], R[0][0])

In [21]:
view(atoms, viewer="x3d")

In [26]:
handedness = np.tile(np.array([1, -1,]), len(R)//2)

In [29]:
out_data = {"R": R, "Z": Z, "handedness": handedness}
np.savez("handedness.npz", **out_data)